# Emotional Classification CNN project

This project is to help me learn about how CNN's work and how do i implement one - from a collection of images to an actual working architecture.

The dataset is from Kaggle and is of a bunch of different faces grouped into different emotions:
  - angry
  - disgusted
  - fearful
  - happy
  - neutral
  - sad
  - surprised


Initially I used VScode as my IDE however i didn't have good GPU support so i chose Colab instead.

### 1. Upload files to Colab

In [ ]:
from google.colab import files
uploaded = files.upload()  # This will open a file picker to upload files

Saving faces.zip to faces.zip


In [ ]:
!unzip -q faces.zip

### 2. Import required modules

In [ ]:
import pandas as pd
import torch                       # core PyTorch library
import torch.nn as nn              # neural network modules (layers, loss functions)
import torch.nn.functional as F
import torch.optim as optim        # optimizers like SGD, Adam
from torch.utils.data import DataLoader  # for batching and shuffling data
from torchvision import datasets, transforms  # popular vision datasets and image transforms
import matplotlib.pyplot as plt    # to plot loss, accuracy, images
import numpy as np                 # numerical operations (optional but handy)
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from torch.utils.data import Subset
from tqdm import tqdm

### 3. Converts images into necessary format and then into tensors, then uses Dataloaders


In [ ]:
print("Script started")

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),        # Converts images to grayscale (1 channel).
    transforms.Resize((64, 64)),                         # Resizes all images to 64x64 pixels.
    transforms.RandomHorizontalFlip(),                   # Randomly flip images horizontally (for data augmentation).
    transforms.RandomRotation(10),                       # Randomly rotate images by up to ±10 degrees.
    transforms.ToTensor(),                               # Convert images to PyTorch tensors (shape: [C, H, W]).
    transforms.Normalize((0.5,), (0.5,))                  # Normalize tensor values: mean=0.5, std=0.5.
])

train_dataset = ImageFolder(root='faces/train', transform=transform) # Loads the training data from folders
test_dataset = ImageFolder(root='faces/test', transform=transform)

#train_dataset_small = Subset(train_dataset, range(100))
#train_loader = DataLoader(train_dataset_small, batch_size=32, shuffle=True, num_workers=0)
# above code: uses only the first 100 training samples, useful for debugging or fast testing

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True) # Wraps datasets into DataLoaders for batching and shuffling
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

images, labels = next(iter(train_loader))
print(images.shape)


Script started
torch.Size([32, 1, 64, 64])


### 4. CNN architecture

In [ ]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(128 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 7)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 128 * 8 * 8)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

### 5. Training loop

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(50):
    running_loss = 0.0
    model.train()
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        loop.set_postfix(loss=running_loss / len(train_loader))


Using device: cuda


Epoch 50: 100%|██████████| 898/898 [00:27<00:00, 33.13it/s, loss=0.776]


### 6. Evaluation step

In [ ]:
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 61.20%


## Conclusion

The Test accuracy is 61.2%. To improve the model, here are some changes I would make to the code:
  - Have more transformations on the images so model has more data to work from
  - play around with the number of epochs
  - play around with learning rate
  - early stopping?
  - Regularization techniques ?